In [1]:
# Dependencies
import pandas as pd

from bs4 import BeautifulSoup
import requests
import pymongo

from sqlalchemy import create_engine
from config import username
from config import password

# Datasource 1

In [2]:
# CSV for 1)	Data source 1 – Alphabetical List of Western Australian Schools (XLSX)

# Store CSV into df
csv_file = "Resources/WASchoolsList_cleaned.csv"
WASchools_df = pd.read_csv(csv_file)
WASchools_df.head()

,Code,School Name,Street,Suburb,State,Postcode,Postal Street,Postal Suburb,Postal State,Postal Postcode,...,Y06,UPR,Y07,Y08,Y09,Y10,Y11,Y12,USE,Total Students
0,5001,ADAM ROAD PRIMARY SCHOOL,HOTCHIN STREET,SOUTH BUNBURY,WA,6230,HOTCHIN STREET,SOUTH BUNBURY,WA,6230,...,45.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,446
1,1463,AL-AMEEN COLLEGE,57 SOUTHGATE ROAD,LANGFORD,WA,6147,57 SOUTHGATE ROAD,LANGFORD,WA,6147,...,96.0,NaN,83.0,79.0,66.0,67.0,48.0,45.0,NaN,"1,136"
2,1397,AL-HIDAYAH ISLAMIC SCHOOL,CNR HEDLEY ST/NYAMUP WAY,BENTLEY,WA,6102,CNR HEDLEY ST/NYAMUP WAY,BENTLEY,WA,6102,...,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202
3,8001,ALBANY COMMUNITY KINDERGARTEN,136 SERPENTINE ROAD,ALBANY,WA,6330,PO BOX 5670,ALBANY,WA,6332,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29
4,5002,ALBANY PRIMARY SCHOOL,SUFFOLK STREET,ALBANY,WA,6330,SUFFOLK STREET,ALBANY,WA,6330,...,58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,426


In [3]:
# look at all the columns available
WASchools_df.columns

Index(['Code', 'School Name', 'Street', 'Suburb', 'State', 'Postcode',
       'Postal Street', 'Postal Suburb', 'Postal State', 'Postal Postcode',
       'Latitude', 'Longitude', 'Phone', 'Education Region',
       'Broad Classification', 'Classification Group', 'Low Year', 'High Year',
       'KIN', 'PPR', 'Y01', 'Y02', 'Y03', 'Y04', 'Y05', 'Y06', 'UPR', 'Y07',
       'Y08', 'Y09', 'Y10', 'Y11', 'Y12', 'USE', 'Total Students'],
      dtype='object')

In [4]:
# create new data with only relevant columns
new_WASchools_df = WASchools_df[['Classification Group','School Name','Low Year','High Year','Y12','Total Students','Education Region','Latitude','Longitude']].copy()
new_WASchools_df.head()


,Classification Group,School Name,Low Year,High Year,Y12,Total Students,Education Region,Latitude,Longitude
0,PRIMARY SCHOOLS,ADAM ROAD PRIMARY SCHOOL,KIN,Y06,NaN,446,SOUTH WEST,-33.357951,115.635371
1,NON-GOVERNMENT,AL-AMEEN COLLEGE,KIN,Y12,45.0,"1,136",SOUTH METROPOLITAN,-32.046056,115.937158
2,NON-GOVERNMENT,AL-HIDAYAH ISLAMIC SCHOOL,PPR,Y06,NaN,202,SOUTH METROPOLITAN,-32.010358,115.911564
3,COMMUNITY KINDERGARTENS,ALBANY COMMUNITY KINDERGARTEN,KIN,KIN,NaN,29,SOUTH WEST,-35.021460,117.880987
4,PRIMARY SCHOOLS,ALBANY PRIMARY SCHOOL,KIN,Y06,NaN,426,SOUTH WEST,-35.018212,117.891611


In [5]:
# remove instances where Y12 is NaN - i.e. dataset is only for secondary schools with Y12 students in it
nan_value = float("NaN")
new_WASchools_df.replace("", nan_value, inplace=True)
new_WASchools_df.dropna(subset = ["Y12"], inplace=True)

# create id
new_WASchools_df.insert(0, 'School_ID', range(0, 0 + len(new_WASchools_df)))

# rename columns
new_WASchools_df = pd.DataFrame(new_WASchools_df).rename(columns = {'School Name':'School_Name'})
new_WASchools_df = pd.DataFrame(new_WASchools_df).rename(columns = {'Classification Group':'Classification_Group'})

new_WASchools_df.head()


,School_ID,Classification_Group,School_Name,Low Year,High Year,Y12,Total Students,Education Region,Latitude,Longitude
1,0,NON-GOVERNMENT,AL-AMEEN COLLEGE,KIN,Y12,45.0,"1,136",SOUTH METROPOLITAN,-32.046056,115.937158
5,1,EDUCATION SUPPORT,ALBANY SECONDARY EDUCATION SUPPORT CENTRE,Y07,Y12,17.0,63,SOUTH WEST,-34.990749,117.866867
6,2,SECONDARY SCHOOLS,ALBANY SENIOR HIGH SCHOOL,Y07,Y12,130.0,958,SOUTH WEST,-35.020448,117.891773
8,3,NON-GOVERNMENT,ALKIMOS BAPTIST COLLEGE,KIN,Y12,12.0,298,NORTH METROPOLITAN,-31.660328,115.710178
12,4,NON-GOVERNMENT,ALL SAINTS' COLLEGE,KIN,Y12,144.0,"1,340",SOUTH METROPOLITAN,-32.049470,115.867023


In [6]:
new_WASchools_df.loc[new_WASchools_df['School_Name'] == "APPLECROSS SENIOR HIGH SCHOOL"]

,School_ID,Classification_Group,School_Name,Low Year,High Year,Y12,Total Students,Education Region,Latitude,Longitude
21,6,SECONDARY SCHOOLS,APPLECROSS SENIOR HIGH SCHOOL,Y07,Y12,286.0,"1,731",SOUTH METROPOLITAN,-32.029376,115.83509


# Datasource 3

In [7]:
# <!-- URL for Data source 3 – WA School Ranking – 2020 -->
url = 'https://bettereducation.com.au/results/wa/wace.aspx'

# <!-- read table -->
tables = pd.read_html(url)

# insert table into a df and drop the first column
df_2020 = tables[0].drop(['Unnamed: 0'],axis=1)

# insert year
df_2020['Year'] = 2020

url = 'https://bettereducation.com.au/results/wa/wace.aspx?yr=2019'
tables = pd.read_html(url)
df_2019 = tables[0].drop(['Unnamed: 0'],axis=1)
df_2019['Year'] = 2019

url = 'https://bettereducation.com.au/results/wa/wace.aspx?yr=2018'
tables = pd.read_html(url)
df_2018 = tables[0].drop(['Unnamed: 0'],axis=1)
df_2018['Year'] = 2018

url = 'https://bettereducation.com.au/results/wa/wace.aspx?yr=2017'
tables = pd.read_html(url)
df_2017 = tables[0].drop(['Unnamed: 0'],axis=1)
df_2017['Year'] = 2017

url = 'https://bettereducation.com.au/results/wa/wace.aspx?yr=2016'
tables = pd.read_html(url)
df_2016 = tables[0].drop(['Unnamed: 0'],axis=1)
df_2016['Year'] = 2016


In [8]:
df= df_2020.append(df_2019).append(df_2018).append(df_2017).append(df_2016)
df



,Better Education Rank,School,Median ATAR,No. eligible Yr 12 students,No. Students with an ATAR,% students with an ATAR,Trend / Compare,Year
0,1,Perth Modern School,97.55,242,242,100.00,Trend / Compare,2020
1,2,St Hilda's Anglican School for Girls,92.70,147,134,91.16,Trend / Compare,2020
2,3,Christ Church Grammar School,92.50,190,175,92.11,Trend / Compare,2020
3,4,Penrhos College,90.65,135,116,85.93,Trend / Compare,2020
4,5,Methodist Ladies' College,90.55,126,112,88.89,Trend / Compare,2020
...,...,...,...,...,...,...,...,...
138,139,Thornlie Senior High School,58.45,125,35,28.00,Trend / Compare,2016
139,140,Lakeland Senior High School,57.70,89,27,30.34,Trend / Compare,2016
140,141,Newton Moore Senior High School,56.95,90,35,38.89,Trend / Compare,2016
141,142,Foundation Christian College,50.65,35,26,74.29,Trend / Compare,2016


In [9]:
cols = list(df.columns)
cols

['Better Education Rank',
 'School',
 'Median ATAR',
 'No. eligible Yr 12 students',
 'No. Students with an ATAR',
 '% students with an ATAR',
 'Trend / Compare',
 'Year']

In [10]:
cols = list(df.columns)
# rename columns to something more descriptive 
cols[0] = "ATAR_Ranking"
# rename School to School Name to tie in with Data Source 2
cols[1] = "School_Name"
cols[2] = "Median_ATAR"
cols[3] = "Eligible_Y12_Students"
cols[4] = "Students_with_an_ATAR"
cols[5] = "Pct_students_with_an_ATAR"
cols[6] = "Trend"
cols[7] = "Year"

df.columns = cols

df = df[ ['Year'] + [ col for col in df.columns if col != 'Year' ] ]
# make the school name uppercase to enable easier merge for end user
# df['School_Name'] = df['School_Name'].str.upper()

df.head()

,Year,ATAR_Ranking,School_Name,Median_ATAR,Eligible_Y12_Students,Students_with_an_ATAR,Pct_students_with_an_ATAR,Trend
0,2020,1,Perth Modern School,97.55,242,242,100.00,Trend / Compare
1,2020,2,St Hilda's Anglican School for Girls,92.70,147,134,91.16,Trend / Compare
2,2020,3,Christ Church Grammar School,92.50,190,175,92.11,Trend / Compare
3,2020,4,Penrhos College,90.65,135,116,85.93,Trend / Compare
4,2020,5,Methodist Ladies' College,90.55,126,112,88.89,Trend / Compare


In [11]:
df=df.drop(['Trend'],axis = 1)

In [12]:
# sort
df = df.sort_values(["ATAR_Ranking", "Year"], ascending = (True, False))
df['School_Name'] = df['School_Name'].str.upper()
print(df)

     Year  ATAR_Ranking                                        School_Name  \
0    2020             1                                PERTH MODERN SCHOOL   
0    2019             1                                PERTH MODERN SCHOOL   
0    2018             1                                PERTH MODERN SCHOOL   
0    2017             1                                PERTH MODERN SCHOOL   
0    2016             1                                PERTH MODERN SCHOOL   
..    ...           ...                                                ...   
144  2017           145                             SOUTHERN RIVER COLLEGE   
145  2018           146                                  DALYELLUP COLLEGE   
145  2017           146                         HEDLAND SENIOR HIGH SCHOOL   
146  2018           147                      ROCKINGHAM SENIOR HIGH SCHOOL   
147  2018           148  WESTERN AUSTRALIAN COLLEGE OF AGRICULTURE (CUN...   

     Median_ATAR  Eligible_Y12_Students  Students_with_an_ATAR 

In [13]:
# merge in school_id
secondary_schools_df = pd.merge(df, new_WASchools_df, on="School_Name", how = "left")
secondary_schools_df.head()

,Year,ATAR_Ranking,School_Name,Median_ATAR,Eligible_Y12_Students,Students_with_an_ATAR,Pct_students_with_an_ATAR,School_ID,Classification_Group,Low Year,High Year,Y12,Total Students,Education Region,Latitude,Longitude
0,2020,1,PERTH MODERN SCHOOL,97.55,242,242,100.00,224.0,SECONDARY SCHOOLS,Y07,Y12,247.0,"1,447",NORTH METROPOLITAN,-31.945066,115.836396
1,2019,1,PERTH MODERN SCHOOL,96.75,241,238,98.76,224.0,SECONDARY SCHOOLS,Y07,Y12,247.0,"1,447",NORTH METROPOLITAN,-31.945066,115.836396
2,2018,1,PERTH MODERN SCHOOL,97.00,240,236,98.33,224.0,SECONDARY SCHOOLS,Y07,Y12,247.0,"1,447",NORTH METROPOLITAN,-31.945066,115.836396
3,2017,1,PERTH MODERN SCHOOL,95.90,223,222,99.55,224.0,SECONDARY SCHOOLS,Y07,Y12,247.0,"1,447",NORTH METROPOLITAN,-31.945066,115.836396
4,2016,1,PERTH MODERN SCHOOL,95.55,222,220,99.10,224.0,SECONDARY SCHOOLS,Y07,Y12,247.0,"1,447",NORTH METROPOLITAN,-31.945066,115.836396


In [14]:
# check if there's any bad data that didn't merge
secondary_schools_df[secondary_schools_df['School_ID'].isnull()]

,Year,ATAR_Ranking,School_Name,Median_ATAR,Eligible_Y12_Students,Students_with_an_ATAR,Pct_students_with_an_ATAR,School_ID,Classification_Group,Low Year,High Year,Y12,Total Students,Education Region,Latitude,Longitude
9,2016,2,PRESBYTERIAN LADIES' COLLEGE,92.90,122,110,90.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,2019,5,PRESBYTERIAN LADIES' COLLEGE,90.75,123,116,94.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,2017,8,PRESBYTERIAN LADIES' COLLEGE,89.60,116,111,95.69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,2020,11,PRESBYTERIAN LADIES' COLLEGE,89.20,109,97,88.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,2018,20,PRESBYTERIAN LADIES' COLLEGE,86.90,127,115,90.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121,2016,24,ST STEPHEN'S SCHOOL (CARRAMAR CAMPUS),85.10,102,67,65.69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
147,2018,30,AUSTRALIAN ISLAMIC COLLEGE (KEWDALE),84.85,96,50,52.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
148,2017,30,GREAT SOUTHERN GRAMMAR SCHOOL,85.00,83,59,71.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168,2018,34,ST STEPHEN'S SCHOOL (CARRAMAR),84.40,97,49,50.52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
181,2019,37,"CHRISTIAN BROTHERS COLLEGE,FREMANTLE,WA,6160",82.30,126,66,52.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
secondary_schools_df = secondary_schools_df.reset_index(drop=True)
secondary_schools_df

,Year,ATAR_Ranking,School_Name,Median_ATAR,Eligible_Y12_Students,Students_with_an_ATAR,Pct_students_with_an_ATAR,School_ID,Classification_Group,Low Year,High Year,Y12,Total Students,Education Region,Latitude,Longitude
0,2020,1,PERTH MODERN SCHOOL,97.55,242,242,100.00,224.0,SECONDARY SCHOOLS,Y07,Y12,247.0,"1,447",NORTH METROPOLITAN,-31.945066,115.836396
1,2019,1,PERTH MODERN SCHOOL,96.75,241,238,98.76,224.0,SECONDARY SCHOOLS,Y07,Y12,247.0,"1,447",NORTH METROPOLITAN,-31.945066,115.836396
2,2018,1,PERTH MODERN SCHOOL,97.00,240,236,98.33,224.0,SECONDARY SCHOOLS,Y07,Y12,247.0,"1,447",NORTH METROPOLITAN,-31.945066,115.836396
3,2017,1,PERTH MODERN SCHOOL,95.90,223,222,99.55,224.0,SECONDARY SCHOOLS,Y07,Y12,247.0,"1,447",NORTH METROPOLITAN,-31.945066,115.836396
4,2016,1,PERTH MODERN SCHOOL,95.55,222,220,99.10,224.0,SECONDARY SCHOOLS,Y07,Y12,247.0,"1,447",NORTH METROPOLITAN,-31.945066,115.836396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,2017,145,SOUTHERN RIVER COLLEGE,55.40,96,24,25.00,259.0,SECONDARY SCHOOLS,Y07,Y12,116.0,970,SOUTH METROPOLITAN,-32.085598,115.979918
714,2018,146,DALYELLUP COLLEGE,51.90,70,29,41.43,72.0,SECONDARY SCHOOLS,Y07,Y12,107.0,749,SOUTH WEST,-33.403855,115.625767
715,2017,146,HEDLAND SENIOR HIGH SCHOOL,52.45,90,29,32.22,123.0,SECONDARY SCHOOLS,Y07,Y12,105.0,999,PILBARA,-20.398789,118.598616
716,2018,147,ROCKINGHAM SENIOR HIGH SCHOOL,47.90,98,22,22.45,238.0,SECONDARY SCHOOLS,Y07,Y12,123.0,"1,030",SOUTH METROPOLITAN,-32.284952,115.735520


In [16]:
# replace bad data
# secondary_schools_df['School_Name'] = df['School_Name'].replace(
#     {"PRESBYTERIAN LADIES' COLLEGE":"PRESBYTERIAN LADIES COLLEGE"
#     # ,
#     # "CHRISTIAN BROTHERS COLLEGE,FREMANTLE,WA,6160":"CHRISTIAN BROTHERS' COLLEGE",
#     # "CAREY BAPTIST COLLEGE,HARRISDALE,WA,6112":"CAREY BAPTIST COLLEGE",
#     # "AUSTRALIAN ISLAMIC COLLEGE (KEWDALE)":"AUSTRALIAN ISLAMIC COLLEGE - KEWDALE"
#     })


In [17]:
# secondary_schools_df[secondary_schools_df['School_ID'].isnull()]
# secondary_schools_df.head()

In [18]:
# drop School_ID for later remerge for clean data
# secondary_schools_df=secondary_schools_df.drop(["School_ID"],axis=1)
# secondary_schools_df.head()

In [19]:
# merge in school_id
# secondary_schools_df2 = pd.merge(new_WASchools_df,secondary_schools_df, on="School_Name", how = "right").drop(["Classification_Group","School_Name","Y12"],axis=1)
# secondary_schools_df2.head()

In [20]:
# check if there's any bad data that didn't merge
# secondary_schools_df2[secondary_schools_df2['School_ID'].isnull()]

In [21]:
secondary_schools_df

,Year,ATAR_Ranking,School_Name,Median_ATAR,Eligible_Y12_Students,Students_with_an_ATAR,Pct_students_with_an_ATAR,School_ID,Classification_Group,Low Year,High Year,Y12,Total Students,Education Region,Latitude,Longitude
0,2020,1,PERTH MODERN SCHOOL,97.55,242,242,100.00,224.0,SECONDARY SCHOOLS,Y07,Y12,247.0,"1,447",NORTH METROPOLITAN,-31.945066,115.836396
1,2019,1,PERTH MODERN SCHOOL,96.75,241,238,98.76,224.0,SECONDARY SCHOOLS,Y07,Y12,247.0,"1,447",NORTH METROPOLITAN,-31.945066,115.836396
2,2018,1,PERTH MODERN SCHOOL,97.00,240,236,98.33,224.0,SECONDARY SCHOOLS,Y07,Y12,247.0,"1,447",NORTH METROPOLITAN,-31.945066,115.836396
3,2017,1,PERTH MODERN SCHOOL,95.90,223,222,99.55,224.0,SECONDARY SCHOOLS,Y07,Y12,247.0,"1,447",NORTH METROPOLITAN,-31.945066,115.836396
4,2016,1,PERTH MODERN SCHOOL,95.55,222,220,99.10,224.0,SECONDARY SCHOOLS,Y07,Y12,247.0,"1,447",NORTH METROPOLITAN,-31.945066,115.836396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,2017,145,SOUTHERN RIVER COLLEGE,55.40,96,24,25.00,259.0,SECONDARY SCHOOLS,Y07,Y12,116.0,970,SOUTH METROPOLITAN,-32.085598,115.979918
714,2018,146,DALYELLUP COLLEGE,51.90,70,29,41.43,72.0,SECONDARY SCHOOLS,Y07,Y12,107.0,749,SOUTH WEST,-33.403855,115.625767
715,2017,146,HEDLAND SENIOR HIGH SCHOOL,52.45,90,29,32.22,123.0,SECONDARY SCHOOLS,Y07,Y12,105.0,999,PILBARA,-20.398789,118.598616
716,2018,147,ROCKINGHAM SENIOR HIGH SCHOOL,47.90,98,22,22.45,238.0,SECONDARY SCHOOLS,Y07,Y12,123.0,"1,030",SOUTH METROPOLITAN,-32.284952,115.735520


In [23]:
secondary_schools_df.to_csv (r'C:\Users\user_491721\Desktop\Analytics Bootcamp\Projects\Project 3\static\schools.csv', index = False, header=True)

In [22]:
secondary_schools_df.columns

Index(['Year', 'ATAR_Ranking', 'School_Name', 'Median_ATAR',
       'Eligible_Y12_Students', 'Students_with_an_ATAR',
       'Pct_students_with_an_ATAR', 'School_ID', 'Classification_Group',
       'Low Year', 'High Year', 'Y12', 'Total Students', 'Education Region',
       'Latitude', 'Longitude'],
      dtype='object')

In [132]:
# convert all numeric to string
string_schools_df = secondary_schools_df
string_schools_df['Year'] = string_schools_df['Year'].apply(str)
string_schools_df['ATAR_Ranking'] = string_schools_df['ATAR_Ranking'].apply(str)
string_schools_df['Median_ATAR'] = string_schools_df['Median_ATAR'].apply(str)
string_schools_df['Eligible_Y12_Students'] = string_schools_df['Eligible_Y12_Students'].apply(str)
string_schools_df['Students_with_an_ATAR'] = string_schools_df['Students_with_an_ATAR'].apply(str)
string_schools_df['Pct_students_with_an_ATAR'] = string_schools_df['Pct_students_with_an_ATAR'].apply(str)
string_schools_df['Y12'] = string_schools_df['Y12'].apply(str)
string_schools_df['Total Students'] = string_schools_df['Total Students'].apply(str)
string_schools_df['Latitude'] = string_schools_df['Latitude'].apply(str)
string_schools_df['Longitude'] = string_schools_df['Longitude'].apply(str)

string_schools_df.dtypes 


Year                         object
ATAR_Ranking                 object
School_Name                  object
Median_ATAR                  object
Eligible_Y12_Students        object
Students_with_an_ATAR        object
Pct_students_with_an_ATAR    object
Classification_Group         object
Low Year                     object
High Year                    object
Y12                          object
Total Students               object
Education Region             object
Latitude                     object
Longitude                    object
dtype: object

In [133]:
# Convert to list for data html

school_list = string_schools_df.values.tolist()
print(school_list)

[['2020', '1', 'PERTH MODERN SCHOOL', '97.55', '242', '242', '100.0', 'SECONDARY SCHOOLS', 'Y07', 'Y12', '247.0', '1,447', 'NORTH METROPOLITAN', '-31.94506602', '115.8363958'], ['2019', '1', 'PERTH MODERN SCHOOL', '96.75', '241', '238', '98.76', 'SECONDARY SCHOOLS', 'Y07', 'Y12', '247.0', '1,447', 'NORTH METROPOLITAN', '-31.94506602', '115.8363958'], ['2018', '1', 'PERTH MODERN SCHOOL', '97.0', '240', '236', '98.33', 'SECONDARY SCHOOLS', 'Y07', 'Y12', '247.0', '1,447', 'NORTH METROPOLITAN', '-31.94506602', '115.8363958'], ['2017', '1', 'PERTH MODERN SCHOOL', '95.9', '223', '222', '99.55', 'SECONDARY SCHOOLS', 'Y07', 'Y12', '247.0', '1,447', 'NORTH METROPOLITAN', '-31.94506602', '115.8363958'], ['2016', '1', 'PERTH MODERN SCHOOL', '95.55', '222', '220', '99.1', 'SECONDARY SCHOOLS', 'Y07', 'Y12', '247.0', '1,447', 'NORTH METROPOLITAN', '-31.94506602', '115.8363958'], ['2020', '2', "ST HILDA'S ANGLICAN SCHOOL FOR GIRLS", '92.7', '147', '134', '91.16', 'NON-GOVERNMENT', 'KIN', 'Y12', '129.

# LOAD

In [69]:
rds_connection_string = f"{username}:{password}@localhost:5432/schools_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [70]:
# check for tables - NOTE this is created in pgadmin before hand
engine.table_names()

<ipython-input-70-2caba68b8947>:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


[]

In [ ]:
# note - delete all tables in pg admin schools_db before running the next step

In [80]:
# NOTE - DON'T RUN MORE THAN ONCE BECUASE IT APPENDS

# Dataset 1
new_WASchools_df2.to_sql(name='wa_schools_y12', con=engine, if_exists='append', index=False)

# Dataset 2
classification_df.to_sql(name='wa_schools_classification', con=engine, if_exists='append', index=False)

# Dataset 3
applecross_df2.to_sql(name='applecross_secondary_schools', con=engine, if_exists='append', index=False)

# Dataset 4
secondary_schools_df2.to_sql(name='secondary_schools_ranking', con=engine, if_exists='append', index=False)

In [81]:
# confirm Dataset 1 has been added
pd.read_sql_query('select * from wa_schools_y12', con=engine).head()

,School_ID,School_Name,Y12,Classification_ID
0,0,AL-AMEEN COLLEGE,45.0,0
1,1,ALBANY SECONDARY EDUCATION SUPPORT CENTRE,17.0,1
2,2,ALBANY SENIOR HIGH SCHOOL,130.0,2
3,3,ALKIMOS BAPTIST COLLEGE,12.0,0
4,4,ALL SAINTS' COLLEGE,144.0,0


In [82]:
# confirm Dataset 2 has been added
pd.read_sql_query('select * from wa_schools_classification', con=engine).head()

,Classification_ID,Classification_Group
0,0,NON-GOVERNMENT
1,1,EDUCATION SUPPORT
2,2,SECONDARY SCHOOLS
3,3,K-12 SCHOOLS
4,4,DISTRICT HIGH SCHOOLS


In [83]:
# confirm Dataset 3 has been added
pd.read_sql_query('select * from applecross_secondary_schools', con=engine).head()

,School_ID
0,6
1,7
2,244
3,63
4,188


In [84]:
# confirm Dataset 4 has been added
pd.read_sql_query('select * from secondary_schools_ranking', con=engine).head()

,School_ID,2020_ATAR_Secondary_School_Ranking,Median_ATAR
0,224,1,97.55
1,266,2,92.70
2,52,3,92.50
3,221,4,90.65
4,192,5,90.55
